**Chapter 4: Docker Basics**

With your development environment operational, we now explore the foundational technology that enables modern CI/CD: **containerization**. Docker transformed software deployment by packaging applications with their dependencies into portable, consistent units. This chapter demystifies containers from the ground up—explaining not just *how* to use Docker, but *why* it solves fundamental deployment challenges. By the end, you will understand container architecture, execute your first containers, and grasp the lifecycle management principles that underpin every Docker-based CI/CD pipeline.

---

### 4.1 What are Containers?

A **container** is a lightweight, standalone, executable package of software that includes everything needed to run an application: code, runtime, system tools, system libraries, and settings. Unlike traditional deployment where applications depend on the host operating system's configuration, containers bundle dependencies to ensure consistent behavior across any environment.

#### The Problem Containers Solve

Consider the traditional deployment scenario: A developer writes a Python application using Python 3.11 on macOS. They deploy to a production server running Python 3.8 on Ubuntu. The application fails because:
- The Python version differs (3.11 vs 3.8)
- A system library (`libssl`) has a different version
- The file path structure differs (`/Users/dev` vs `/home/ubuntu`)
- Environment variables are missing or different

**Container Solution:** The developer packages the application with Python 3.11, `libssl` at the exact required version, and all dependencies into a container image. This image runs identically on macOS, Ubuntu, or Windows because it brings its own isolated environment.

#### Container Technology Under the Hood

Containers are not virtual machines, nor are they simple chroot jails. They leverage specific Linux kernel features (also supported by Windows and macOS through abstraction layers):

**Namespaces** provide isolation by restricting what a container can see:
- **PID Namespace:** Process isolation (PID 1 in container is different from host PID 1)
- **Network Namespace:** Separate network stack (interfaces, routing tables)
- **Mount Namespace:** Isolated filesystem view
- **UTS Namespace:** Unique hostname
- **IPC Namespace:** Isolated inter-process communication
- **User Namespace:** Maps container root to non-root host user (security)

**Control Groups (cgroups)** limit resource usage:
- CPU shares and quotas
- Memory limits (hard and soft)
- Block I/O bandwidth
- Device access control

**Union File Systems** enable efficient image layering:
- OverlayFS (modern standard)
- AUFS (legacy)
- devicemapper (older Docker versions)

**Capabilities** provide fine-grained privilege control:
- Containers run with dropped capabilities by default (no `CAP_SYS_ADMIN`, etc.)
- Only necessary kernel permissions are granted

#### Container Standards (OCI)

The **Open Container Initiative (OCI)** maintains industry standards ensuring container portability:
- **Runtime Specification (`runtime-spec`):** How to run a container (implemented by `runc`, `crun`)
- **Image Specification (`image-spec`):** Format for container images (manifests, layers, configurations)

Docker complies with OCI standards, meaning images built with Docker can run on containerd, Podman, or Kubernetes without modification.

**Key Takeaway:** Containers are **standardized, isolated process environments** that package applications with their complete dependency tree, ensuring behavioral consistency across development, staging, and production environments.

---

### 4.2 Docker Architecture Overview

Docker uses a **client-server architecture** consisting of three main components that communicate via REST APIs. Understanding this architecture is essential for debugging, security configuration, and CI/CD pipeline design.

#### The Docker Daemon (`dockerd`)

The Docker daemon is the persistent background process that manages Docker objects:
- **Image Management:** Building, pulling, pushing, and storing images
- **Container Runtime:** Creating, starting, stopping, and monitoring containers
- **Network Management:** Creating virtual networks, assigning IPs, managing firewall rules
- **Volume Management:** Managing persistent storage locations
- **API Endpoint:** Listening on `unix:///var/run/docker.sock` (Linux) or named pipes/named sockets (Windows/macOS)

**Configuration:**
The daemon is configured via `/etc/docker/daemon.json` (Linux) or Docker Desktop settings (GUI):

```json
{
  "hosts": ["unix:///var/run/docker.sock", "tcp://0.0.0.0:2376"],
  "tls": true,
  "tlscacert": "/etc/docker/ca.pem",
  "tlscert": "/etc/docker/server-cert.pem",
  "tlskey": "/etc/docker/server-key.pem",
  "tlsverify": true,
  "storage-driver": "overlay2",
  "log-driver": "json-file",
  "log-opts": {
    "max-size": "10m",
    "max-file": "3"
  }
}
```

**Security Note:** Exposing the Docker TCP socket (`tcp://0.0.0.0:2375`) without TLS grants root access to anyone who can connect. Always use TLS or Unix sockets for production.

#### The Docker Client (`docker`)

The Docker client is the command-line interface (CLI) users interact with. It sends commands to the daemon via the REST API.

**Command Structure:**
```bash
docker [command] [subcommand] [options] [arguments]
```

Examples:
- `docker run` → Client sends POST to `/containers/create` then `/containers/{id}/start`
- `docker build` → Client sends build context to daemon, daemon executes build

**Context Management:**
Modern Docker supports multiple contexts for switching between different daemons (local, remote, cloud):

```bash
# Create context for remote CI/CD builder
docker context create ci-builder --docker "host=ssh://ci-server.example.com"

# Switch contexts
docker context use ci-builder

# List contexts
docker context ls
```

#### Docker Registries

A **registry** stores Docker images. The architecture supports a federated model:

**Docker Hub:** The default public registry (docker.io)
- Contains official images (nginx, python, ubuntu)
- Supports private repositories
- Rate limits apply for unauthenticated pulls (100 pulls per 6 hours)

**Private Registries:**
- **Cloud:** Amazon ECR, Google GCR, Azure ACR
- **Self-hosted:** Harbor, Docker Registry (open source), GitLab Container Registry

**Repository Naming Convention:**
```
[registry_host/][namespace/]repository:tag

# Examples:
nginx:latest                    # Docker Hub official image
myuser/myapp:v1.0.0            # Docker Hub user repository
gcr.io/project-id/api:sha-abc  # Google Container Registry
123456789012.dkr.ecr.us-east-1.amazonaws.com/app:1.0  # Amazon ECR
```

#### Container Runtime Interface (CRI)

While Docker historically used its own runtime, modern Kubernetes uses the **Container Runtime Interface (CRI)**, typically implemented by **containerd** or **CRI-O**. Docker Desktop now uses containerd internally for improved resource management and Kubernetes integration.

**Architecture Flow:**
```
User → Docker CLI → Docker Daemon → containerd → runc → Container
```

This abstraction means you can build images with Docker but run them with Podman or Kubernetes without compatibility issues.

**Key Takeaway:** Docker is not a monolithic tool but a **modular platform** consisting of client, daemon, and registry components. Understanding this separation clarifies security boundaries (daemon runs as root, client does not) and enables remote management scenarios essential for CI/CD build farms.

---

### 4.3 Running Your First Container

With architecture understood, we execute practical container operations. These commands form the foundation of every Dockerfile and CI/CD pipeline.

#### The `docker run` Command

`docker run` creates and starts a container in one operation. It is the most complex Docker command with numerous options.

**Basic Syntax:**
```bash
docker run [OPTIONS] IMAGE[:TAG] [COMMAND] [ARG...]
```

**Your First Container:**
```bash
# Run a simple container that prints "Hello World"
docker run hello-world
```

**What happens:**
1. Docker checks for `hello-world:latest` locally
2. If not found, pulls from Docker Hub
3. Creates a new container from the image
4. Runs the default command (prints informational message)
5. Exits (container stops but remains in stopped state)

#### Interactive Containers

Run an interactive Ubuntu container to explore the isolated environment:

```bash
docker run -it ubuntu:22.04 bash
```

**Option Breakdown:**
- `-i` (interactive): Keep STDIN open even if not attached
- `-t` (tty): Allocate a pseudo-TTY (terminal)
- `ubuntu:22.04`: Specific image and tag (not latest)
- `bash`: Override default command to run bash shell

**Inside the container, observe isolation:**
```bash
# Check process list (only sees container processes)
ps aux

# Check hostname (randomly generated container ID)
hostname

# Check network interfaces (isolated from host)
ip addr

# Check disk space (sees only container layers)
df -h

# Exit stops the container
exit
```

#### Background (Detached) Containers

Services (web servers, databases) run in detached mode:

```bash
# Run Nginx in background
docker run -d --name web-server -p 8080:80 nginx:alpine

# Verify it's running
docker ps

# View logs
docker logs web-server

# Stop the container
docker stop web-server
```

**Option Breakdown:**
- `-d` (detached): Run in background
- `--name`: Assign readable name (otherwise random)
- `-p 8080:80`: Port mapping (host:container)

#### Container Resource Constraints

Production containers must have resource limits to prevent noisy neighbor problems:

```bash
docker run -d \
  --name limited-app \
  --memory="512m" \
  --memory-swap="512m" \
  --cpus="1.5" \
  --pids-limit=100 \
  nginx:alpine
```

**Resource Options:**
- `--memory`: Hard memory limit (e.g., `512m`, `2g`)
- `--memory-swap`: Swap limit (set equal to memory to disable swap)
- `--cpus`: CPU quota (1.5 = 1.5 cores)
- `--pids-limit`: Maximum processes/threads (prevents fork bombs)

#### Environment Variables

Inject configuration without modifying images:

```bash
docker run -d \
  -e DATABASE_URL=postgres://db:5432/app \
  -e API_KEY=secret123 \
  -e LOG_LEVEL=debug \
  --env-file ./.env \
  myapp:latest
```

**Best Practice:** Never commit secrets in images. Always inject via environment variables or secrets management (covered in Chapter 10).

**Key Takeaway:** The `docker run` command is your primary interface for container instantiation. Master its options—port mapping, volume mounting, environment variables, and resource limits—as these translate directly to Kubernetes Pod specifications and CI/CD deployment configurations.

---

### 4.4 Container Lifecycle Management

Containers transition through distinct states: created, running, paused, stopped, and deleted. Understanding these states and the commands that trigger transitions is essential for operations and troubleshooting.

#### Container States

```
Created → Running → Paused → Stopped → Deleted
   ↑        ↓         ↓        ↓
   └────────┴─────────┴────────┘
     (Restart policies can return to Running)
```

**Created:** Container configuration exists but process has not started
**Running:** Process is executing
**Paused:** Process frozen (SIGSTOP), memory retained
**Stopped:** Process terminated (SIGTERM/SIGKILL), metadata retained
**Deleted:** Container removed from system

#### Lifecycle Commands

**Creating Without Starting:**
```bash
# Create container configuration
docker create --name my-nginx nginx:alpine

# Verify state (Created, not Running)
docker ps -a
```

**Starting/Stopping:**
```bash
# Start the created container
docker start my-nginx

# Graceful stop (SIGTERM, waits 10s, then SIGKILL)
docker stop my-nginx

# Force kill (SIGKILL, immediate)
docker kill my-nginx

# Restart (stop + start)
docker restart my-nginx
```

**Pause/Unpause:**
```bash
# Freeze container (useful for debugging resource issues)
docker pause my-nginx

# Resume
docker unpause my-nginx
```

**Removing Containers:**
```bash
# Remove stopped container
docker rm my-nginx

# Force remove running container (SIGKILL + remove)
docker rm -f my-nginx

# Remove all stopped containers
docker container prune

# Remove container and its anonymous volumes
docker rm -v my-nginx
```

#### Restart Policies

Define automatic recovery behavior:

```bash
# Always restart (even if manually stopped, except docker stop)
docker run -d --restart always nginx:alpine

# Restart only on failure, with max 3 attempts
docker run -d --restart on-failure:3 myapp

# Restart unless manually stopped
docker run -d --restart unless-stopped nginx:alpine
```

**CI/CD Implication:** Use `unless-stopped` for development environments to prevent automatic restarts during maintenance, and `always` for production services.

#### Inspecting Containers

**Process Information:**
```bash
# Top (processes inside container)
docker top my-nginx

# Stats (resource usage)
docker stats my-nginx --no-stream
```

**Metadata Inspection:**
```bash
# Detailed JSON inspection
docker inspect my-nginx

# Extract specific fields (using Go templates)
docker inspect -f '{{.NetworkSettings.IPAddress}}' my-nginx
docker inspect -f '{{.State.Status}}' my-nginx
docker inspect -f '{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' my-nginx
```

#### Exec into Running Containers

Execute commands in running containers for debugging:

```bash
# Interactive shell
docker exec -it my-nginx sh

# Single command
docker exec my-nginx nginx -t  # Test nginx configuration

# Run as specific user
docker exec -u www-data -it my-nginx sh

# Environment variables
docker exec my-nginx env
```

**Warning:** Changes made via `docker exec` are not persisted to the image. They are lost when the container restarts.

**Key Takeaway:** Container lifecycle management mirrors process management but with additional abstraction. Understanding the distinction between **stopping** (retaining state for debugging) and **removing** (complete cleanup) is crucial for resource management in CI/CD environments where disk space is finite.

---

### 4.5 Working with Docker Images

Images are the **blueprints** for containers—read-only templates containing application code, libraries, and configuration. Understanding image architecture enables optimization and security scanning.

#### Image Architecture

Docker images use a **layered Union File System**:

```
Container Layer (Writable)
└── Image Layer N (Application code)
└── Image Layer 3 (Dependencies)
└── Image Layer 2 (Runtime)
└── Image Layer 1 (Base OS)
└── Image Layer 0 (scratch/empty)
```

Each layer is a set of filesystem changes (diff). Layers are shared between images (deduplication) and cached during builds.

#### Managing Images

**Listing Images:**
```bash
# Local images
docker images
docker image ls

# Filter dangling (untagged) images
docker images -f dangling=true

# Filter by label
docker images -f label=version=1.0
```

**Pulling Images:**
```bash
# Latest tag (avoid in production)
docker pull nginx

# Specific version (recommended)
docker pull nginx:1.25.3-alpine

# Specific digest (immutable, most secure)
docker pull nginx@sha256:abc123...
```

**Removing Images:**
```bash
# By tag
docker rmi nginx:alpine

# By ID
docker rmi a1b2c3d4

# Force remove (even if containers exist)
docker rmi -f nginx:alpine

# Remove all unused images
docker image prune -a
```

#### Image Inspection

**History (Layer Analysis):**
```bash
docker history nginx:alpine
```

Shows each layer's creation command, size, and timestamp. Essential for optimizing image size.

**Detailed Inspection:**
```bash
docker inspect nginx:alpine

# Extract specific config
docker inspect -f '{{.Config.Env}}' nginx:alpine
docker inspect -f '{{.Config.Cmd}}' nginx:alpine
docker inspect -f '{{.RootFS.Layers}}' nginx:alpine
```

#### Saving and Loading Images

For air-gapped environments or backup:

```bash
# Save to tar archive
docker save -o nginx-alpine.tar nginx:alpine

# Compress
gzip nginx-alpine.tar

# Load on another system
docker load -i nginx-alpine.tar.gz

# Alternative: Export container filesystem (loses history)
docker export -o nginx-fs.tar my-nginx
```

**CI/CD Use Case:** Pre-pull and save base images in CI caches to avoid Docker Hub rate limits:

```bash
# In pipeline setup
docker pull node:20-alpine
docker save node:20-alpine -o cache/node.tar
# Cache the tar file between builds
```

#### Multi-Platform Images

Modern CI/CD builds for multiple architectures (AMD64, ARM64):

```bash
# Pull specific platform
docker pull --platform linux/arm64 nginx:alpine

# Buildx for multi-platform (covered in Chapter 8)
docker buildx build --platform linux/amd64,linux/arm64 -t myapp:latest .
```

**Key Takeaway:** Images are **immutable, layered artifacts**. Understanding layer caching is crucial for CI/CD build optimization—each layer that doesn't change avoids rebuild time. Always use specific tags or digests rather than `latest` for reproducible builds.

---

### 4.6 Docker Registry Concepts

Registries store and distribute images. CI/CD pipelines interact heavily with registries—pushing built images and pulling deployment artifacts.

#### Docker Hub

The default registry (`docker.io`):

**Repository Types:**
- **Official:** Curated by Docker (nginx, python, node) - no namespace
- **Verified:** Published by software vendors (microsoft, google)
- **Community:** User-published (username/repo)

**Rate Limits:**
- Anonymous: 100 pulls per 6 hours per IP
- Authenticated (free): 200 pulls per 6 hours
- Pro/Team: Unlimited

**Authentication:**
```bash
# Login (stores credentials in ~/.docker/config.json)
docker login docker.io

# Logout
docker logout docker.io
```

**Security Warning:** `~/.docker/config.json` stores credentials base64-encoded (not encrypted). Use credential helpers (installed in Chapter 3) for secure storage.

#### Tagging Strategy

Tags are mutable pointers to image digests (immutable hashes). A robust tagging strategy is essential for CI/CD:

**Anti-Pattern (Avoid):**
```bash
docker build -t myapp:latest .
docker push myapp:latest
# Overwrites previous latest, loses rollback capability
```

**Recommended Strategy:**
```bash
COMMIT_SHA=$(git rev-parse --short HEAD)
BRANCH=$(git rev-parse --abbrev-ref HEAD)

# Multiple tags for traceability
docker build -t myapp:${COMMIT_SHA} \
             -t myapp:${BRANCH} \
             -t myapp:latest .

docker push myapp:${COMMIT_SHA}  # Immutable, used for deployments
docker push myapp:${BRANCH}      # Mutable, tracks branch head
```

**Semantic Versioning (for releases):**
```bash
VERSION="1.2.3"
docker tag myapp:latest myapp:${VERSION}
docker tag myapp:latest myapp:1.2
docker tag myapp:latest myapp:1
# Push all tags
```

#### Private Registry Operations

**Amazon ECR Example:**
```bash
# Authenticate (token expires every 12 hours)
aws ecr get-login-password --region us-east-1 | \
  docker login --username AWS --password-stdin 123456789012.dkr.ecr.us-east-1.amazonaws.com

# Tag for ECR
docker tag myapp:latest 123456789012.dkr.ecr.us-east-1.amazonaws.com/myapp:latest

# Push
docker push 123456789012.dkr.ecr.us-east-1.amazonaws.com/myapp:latest
```

**Harbor (Open Source Registry):**
Supports vulnerability scanning, RBAC, and replication between registries—ideal for air-gapped CI/CD.

#### Registry Best Practices

1. **Immutable Tags:** Use commit SHA for deployment references; never redeploy `latest`
2. **Lifecycle Policies:** Automatically delete untagged images or images older than N days (ECR supports this)
3. **Content Trust:** Sign images with Docker Content Trust (Notary) or cosign (Sigstore)
4. **Vulnerability Scanning:** Scan on push (built into Harbor, ECR, GCR)
5. **Geo-Replication:** Host registries in multiple regions for faster CI/CD pulls

```bash
# Enable Docker Content Trust (signing)
export DOCKER_CONTENT_TRUST=1
export DOCKER_CONTENT_TRUST_REPOSITORY_PASSPHRASE="supersecret"

# Sign and push
docker push myregistry/myapp:signed-tag
```

**Key Takeaway:** Registries are the **artifact repositories** of CI/CD. Treat them with the same rigor as code repositories—implement retention policies, vulnerability scanning, and immutable references to ensure reliable, secure deployments.

---

### 4.7 Docker vs. Virtual Machines

While both provide isolation, containers and VMs differ fundamentally in architecture, performance characteristics, and use cases. Understanding these differences guides architectural decisions in CI/CD pipelines.

#### Architectural Comparison

**Virtual Machines:**
```
Hardware → Hypervisor → Guest OS (Kernel + Userspace) → App A
                          Guest OS (Kernel + Userspace) → App B
                          Guest OS (Kernel + Userspace) → App C
```

**Containers:**
```
Hardware → Host OS (Shared Kernel) → Container A (Userspace only)
                                   → Container B (Userspace only)
                                   → Container C (Userspace only)
```

#### Detailed Comparison Table

| Aspect | Virtual Machines | Docker Containers |
|--------|-----------------|-------------------|
| **Isolation** | Hardware-level (stronger) | OS-level (process isolation) |
| **Guest OS** | Each VM has full OS (~GBs) | Shares host kernel (~MBs) |
| **Boot Time** | Minutes (full OS boot) | Seconds (process start) |
| **Performance** | Near-native with KVM | Native (no hypervisor overhead) |
| **Density** | Tens per server | Thousands per server |
| **Storage** | Heavy (10GB+ per VM) | Lightweight (shared layers) |
| **Security** | Stronger boundary (kernel exploit affects one VM) | Weaker boundary (kernel exploit affects all containers) |
| **Portability** | Hardware-dependent (VMware, KVM formats differ) | Universal (OCI standard) |
| **Management** | Infrastructure-focused (vCenter) | Application-focused (Kubernetes) |

#### When to Use VMs

Despite container popularity, VMs remain appropriate for:
- **Multi-tenant isolation:** Running untrusted third-party code where kernel escape is unacceptable
- **Legacy applications:** Requiring specific kernel versions or OS features
- **Heavy resource isolation:** Guaranteed CPU/memory boundaries (though containers can achieve this with cgroups)
- **Different OS requirements:** Windows and Linux on same hardware (though Windows containers exist)

#### When to Use Containers

Containers excel in CI/CD scenarios:
- **Microservices:** Hundreds of small services requiring independent deployment
- **Ephemeral workloads:** CI build agents that spin up, execute tests, and terminate
- **Resource efficiency:** Maximizing hardware utilization in cloud environments
- **Consistency:** Identical environment from laptop to production
- **Immutable infrastructure:** Replacing rather than modifying running systems

#### Hybrid Approaches

Modern infrastructure often combines both:

**Kata Containers / Firecracker:** VM-level isolation with container UX
**VMs hosting Kubernetes:** Each VM is a Kubernetes node running dozens of containers
**Nested virtualization:** Containers for development speed, VMs for production isolation in regulated industries

#### Performance Benchmarks

**Startup Time:**
```bash
# VM boot time: 30-120 seconds
# Container start time: 0.5-2 seconds

time docker run --rm alpine echo "hello"
# real 0m0.523s
```

**Resource Overhead:**
```bash
# Alpine Linux container: ~5MB
docker images alpine

# Minimal Ubuntu VM image: ~200MB+
```

**Key Takeaway:** Containers provide **sufficient isolation for most CI/CD and microservice scenarios** with significantly better resource efficiency and startup speed than VMs. Reserve VMs for scenarios requiring strong security boundaries or legacy OS support. In cloud-native CI/CD, containers are the default unit of deployment.

---

### 4.8 When to Use Docker

Docker and containerization solve specific problems brilliantly but introduce unnecessary complexity in others. This section establishes decision frameworks for adopting Docker in your CI/CD pipeline.

#### Ideal Use Cases

**1. Microservices Architecture**
When decomposing monoliths into independently deployable services:
- Each service containerized with its specific dependencies
- Independent scaling (10 instances of service A, 2 of service B)
- Polyglot persistence (PostgreSQL for auth, Redis for cache, MongoDB for logs)

**2. Continuous Integration Pipelines**
Providing consistent build environments:
```yaml
# GitHub Actions using Docker containers for steps
jobs:
  test:
    runs-on: ubuntu-latest
    container:
      image: node:20-alpine  # Exact version every time
    steps:
      - uses: actions/checkout@v4
      - run: npm test
```

**3. Development Environment Standardization**
Eliminating "works on my machine":
- New developers run `docker-compose up` instead of installing Python, Node, PostgreSQL locally
- Database versions pinned (PostgreSQL 14.2, not "whatever is in Homebrew")

**4. Cloud Migration and Portability**
Moving between cloud providers:
- Container image runs identically on AWS ECS, Google GKE, Azure AKS, or on-premises
- No vendor lock-in at the application layer

**5. Legacy Application Modernization**
Wrapping monolithic applications without rewriting:
- Containerize the monolith first
- Gradually extract services while keeping the container interface stable

#### Anti-Patterns (When NOT to Use Docker)

**1. Simple CLI Tools**
For a standalone Python script with no dependencies:
- Overhead: Building, pushing, pulling an image
- Better: Direct execution or virtualenv

**2. Stateful Applications with Heavy I/O**
Databases in containers require careful consideration:
- Container filesystem is ephemeral by default (data lost on restart)
- Network overhead for database connections
- Better: Managed database services (RDS, Cloud SQL) or VMs for high-performance databases

**3. Desktop GUI Applications**
While possible (X11 forwarding), containers add complexity for little benefit for GUI apps.

**4. Situations Requiring Persistent Kernel State**
If your application modifies kernel modules, sysctl settings, or requires specific kernel versions, containers (which share the host kernel) are unsuitable.

#### The CI/CD Decision Matrix

| Factor | Use Docker | Avoid Docker |
|--------|-----------|-------------|
| **Team Size** | >3 developers (consistency) | Solo developer |
| **Deployment Frequency** | Multiple times daily | Monthly releases |
| **Environment Count** | Dev, Staging, Prod (need parity) | Single production server |
| **Service Count** | >3 services | Single monolith |
| **Infrastructure** | Cloud/Kubernetes | Bare metal legacy |

#### Migration Strategy

If adopting Docker for existing CI/CD:

**Phase 1: Containerize the Build (Chapter 5)**
Run existing build scripts inside containers without changing the build logic.

**Phase 2: Containerize the Application (Chapter 6)**
Package the application for deployment, initially treating containers as "lightweight VMs."

**Phase 3: Optimize for Containers (Chapter 9)**
Implement multi-stage builds, health checks, and proper signal handling.

**Phase 4: Orchestrate (Part III)**
Move from `docker run` to Kubernetes for production orchestration.

**Key Takeaway:** Adopt Docker when **environment consistency and deployment frequency** justify the containerization overhead. For simple, stable, single-service applications, the additional complexity may not provide ROI. However, for modern CI/CD practices involving microservices, rapid iteration, and cloud deployment, containers are not merely beneficial—they are essential infrastructure.

---

### Chapter Summary and Preview

In this chapter, you mastered the foundational concepts of containerization. You learned that **containers** are isolated process environments leveraging Linux namespaces and cgroups, distinct from VMs in architecture and performance characteristics. We explored the **Docker architecture**—client, daemon, and registry—and executed your first containers using `docker run` with various options for interactivity, detachment, and resource constraints. You managed the **container lifecycle** through creation, start, stop, and removal operations, and understood the **layered image architecture** that enables efficient storage and caching. We examined **registry operations** including tagging strategies essential for CI/CD traceability, and established decision criteria for when containerization provides value versus introducing unnecessary complexity.

You can now execute containers, inspect their properties, and understand how they fit into the broader software delivery lifecycle. These manual operations—running containers, managing images, and configuring registries—form the building blocks for the automated pipelines we will construct in later chapters.

In **Chapter 5: Dockerfiles - Building Images**, we transition from consuming existing images to creating custom ones. You will learn to write Dockerfiles—the declarative recipes for image construction—mastering instructions like FROM, RUN, COPY, and CMD. We will build images for multiple programming languages, implement multi-stage builds for optimization, and establish security best practices that ensure your containers are production-ready. This is where you gain the power to package your applications consistently, creating the immutable artifacts that flow through your CI/CD pipeline from build to production deployment.